# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path
import os
import matplotlib.pyplot as plt
import geopandas as gpd

# Import API key
from api_keys import geoapify_key
from api_keys import gmaps_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,puerto ayora,-0.7393,-90.3518,20.86,99,19,2.47,EC,2024-07-03
1,1,uturoa,-16.7333,-151.4333,25.84,75,38,10.64,PF,2024-07-03
2,2,adamstown,-25.0660,-130.1015,20.60,62,100,6.11,PN,2024-07-03
3,3,ushuaia,-54.8000,-68.3000,1.81,80,75,6.69,AR,2024-07-03
4,4,ilulissat,69.2167,-51.1000,3.01,100,100,0.51,GL,2024-07-03


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.scatter("Lng", "Lat", frame_width=800, frame_height=500, size="Humidity", by="Humidity", hover_cols="City")
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
final_plot = world.hvplot() * map_plot

# Display the map
final_plot

:Overlay
   .Polygons.I  :Polygons   [x,y]
   .NdOverlay.I :NdOverlay   [Humidity]
      :Scatter   [Lng]   (Lat,Humidity,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
city_ideal = city_data_df.loc[(city_data_df["Max Temp"]>21) & (city_data_df["Max Temp"]<=27) & (city_data_df["Wind Speed"]<=4.5) & (city_data_df["Cloudiness"]==0)]

# Drop any rows with null values
ideal_cities_df = city_ideal.dropna()

# Display sample data
ideal_cities_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
46,46,jamestown,42.0970,-79.2353,21.12,64,0,3.60,US,2024-07-03
58,58,emporia,38.4039,-96.1817,23.11,94,0,1.54,US,2024-07-03
177,177,ambovombe,-25.1667,46.0833,24.17,52,0,3.93,MG,2024-07-03
237,237,ben gardane,33.1378,11.2197,26.93,51,0,3.52,TN,2024-07-03
248,248,kill devil hills,36.0307,-75.6760,22.77,88,0,0.00,US,2024-07-03
253,253,remire-montjoly,4.9167,-52.2667,23.02,100,0,0.00,GF,2024-07-03
335,335,tiznit,29.5833,-9.5000,24.25,51,0,2.05,MA,2024-07-03
355,355,shibetsu,43.6590,145.1320,25.39,57,0,3.09,JP,2024-07-03
422,422,panjab,34.3880,67.0233,24.28,12,0,3.84,AF,2024-07-03
427,427,stephenville,32.2207,-98.2023,26.25,68,0,2.57,US,2024-07-03


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
46,jamestown,US,42.0970,-79.2353,64,
58,emporia,US,38.4039,-96.1817,94,
177,ambovombe,MG,-25.1667,46.0833,52,
237,ben gardane,TN,33.1378,11.2197,51,
248,kill devil hills,US,36.0307,-75.6760,88,
253,remire-montjoly,GF,4.9167,-52.2667,100,
335,tiznit,MA,29.5833,-9.5000,51,
355,shibetsu,JP,43.6590,145.1320,57,
422,panjab,AF,34.3880,67.0233,12,
427,stephenville,US,32.2207,-98.2023,68,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey": geoapify_key
}


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    params["categories"] = "accommodation.hotel"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
jamestown - nearest hotel: DoubleTree Jamestown
emporia - nearest hotel: Budget Host Inn Emporia
ambovombe - nearest hotel: No hotel found
ben gardane - nearest hotel: No hotel found
kill devil hills - nearest hotel: Mariner Days Inn & Suites
remire-montjoly - nearest hotel: Complexe Belova
tiznit - nearest hotel: No hotel found
shibetsu - nearest hotel: 楠旅館
panjab - nearest hotel: Bazar Dan Siah Darah - بازار دان سیاه دره
stephenville - nearest hotel: No hotel found
tsiombe - nearest hotel: No hotel found
inhambane - nearest hotel: Hotel de Inhambane
center - nearest hotel: Quality Inn & Suites


,City,Country,Lat,Lng,Humidity,Hotel Name
46,jamestown,US,42.0970,-79.2353,64,DoubleTree Jamestown
58,emporia,US,38.4039,-96.1817,94,Budget Host Inn Emporia
177,ambovombe,MG,-25.1667,46.0833,52,No hotel found
237,ben gardane,TN,33.1378,11.2197,51,No hotel found
248,kill devil hills,US,36.0307,-75.6760,88,Mariner Days Inn & Suites
253,remire-montjoly,GF,4.9167,-52.2667,100,Complexe Belova
335,tiznit,MA,29.5833,-9.5000,51,No hotel found
355,shibetsu,JP,43.6590,145.1320,57,楠旅館
422,panjab,AF,34.3880,67.0233,12,Bazar Dan Siah Darah - بازار دان سیاه دره
427,stephenville,US,32.2207,-98.2023,68,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
mapb = hotel_df.hvplot.points('Lng', 'Lat', 
    geo=True, tiles = True, alpha=0.70, marker='o', 
    frame_width = 700, frame_height = 500,
    s='Humidity', c='City', 
    hover_alpha=1, 
    hover_cols=['Hotel Name', 'Country']
)
# Display the map
mapb

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)